In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# 資料集路徑
dataset_path = 'fer2013'
train_dir = os.path.join(dataset_path, 'train')
test_dir = os.path.join(dataset_path, 'test')

# 資料預處理與數據增強
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1  # 將訓練數據分為訓練集和驗證集
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,  # 修正為從訓練資料中劃分驗證集
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    subset='validation'
)

# 模型架構設計
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.001)),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 訓練回調函數
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,  # 增大耐心值
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

# 訓練模型
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)

# 測試集驗證
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_generator = test_datagen.flow_from_directory(
    test_dir,  # 修正測試集路徑
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)

# 評估模型
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Found 25841 images belonging to 7 classes.
Found 2868 images belonging to 7 classes.


c:\Users\hoisaline\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\hoisaline\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 0s 636ms/step - accuracy: 0.1938 - loss: 2.4143

c:\Users\hoisaline\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


404/404 ━━━━━━━━━━━━━━━━━━━━ 273s 672ms/step - accuracy: 0.1938 - loss: 2.4137 - val_accuracy: 0.1897 - val_loss: 3.3283 - learning_rate: 1.0000e-04
Epoch 2/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 161s 398ms/step - accuracy: 0.2355 - loss: 2.0907 - val_accuracy: 0.2706 - val_loss: 2.1498 - learning_rate: 1.0000e-04
Epoch 3/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 173s 428ms/step - accuracy: 0.2485 - loss: 2.0498 - val_accuracy: 0.2894 - val_loss: 2.1964 - learning_rate: 1.0000e-04
Epoch 4/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 177s 439ms/step - accuracy: 0.2645 - loss: 2.0248 - val_accuracy: 0.2744 - val_loss: 2.1851 - learning_rate: 1.0000e-04
Epoch 5/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 172s 426ms/step - accuracy: 0.2643 - loss: 1.9913 - val_accuracy: 0.3020 - val_loss: 2.0281 - learning_rate: 1.0000e-04
Epoch 6/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 179s 444ms/step - accuracy: 0.2715 - loss: 1.9553 - val_accuracy: 0.3281 - val_loss: 1.8942 - learning_rate: 1.0000e-04
Epoch 7/20
404/404 ━━━━━━━━━━━━━━━━━━━━ 179s 443ms/

In [6]:
import os
from collections import Counter

# 訓練數據夾路徑
train_classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]

# 計算每個類別的樣本數量
train_counts = {cls: len(os.listdir(os.path.join(train_dir, cls))) for cls in train_classes}

print("訓練集各類別樣本數量：", train_counts)


訓練集各類別樣本數量： {'angry': 3995, 'disgust': 436, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}


In [9]:
import os
from collections import Counter

# 訓練數據夾路徑
test_classes = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]

# 計算每個類別的樣本數量
test_counts = {cls: len(os.listdir(os.path.join(test_dir, cls))) for cls in test_classes}

print("訓練集各類別樣本數量：", test_counts)


訓練集各類別樣本數量： {'angry': 958, 'disgust': 111, 'fear': 1024, 'happy': 1774, 'neutral': 1233, 'sad': 1247, 'surprise': 831}


In [23]:
# 數據增強設置
augment_disgust = ImageDataGenerator(
    rotation_range=40,         # 隨機旋轉
    width_shift_range=0.2,     # 水平平移
    height_shift_range=0.2,    # 垂直平移
    zoom_range=0.2,            # 隨機縮放
    horizontal_flip=True,      # 水平翻轉
    fill_mode='nearest'        # 填充模式
)

# 增強數據保存到新路徑
augmented_path = 'disgust_images/'  # 保存增強後的圖片
disgust_generator = augment_disgust.flow_from_directory(
    'fer2013/train',           # 指向父資料夾
    classes=['disgust'],       # 指定 "disgust" 類別
    target_size=(96, 96),      # 與模型輸入尺寸一致
    batch_size=32,
    save_to_dir=augmented_path,  # 保存增強後圖片
    save_format='jpeg',
    class_mode=None
)

# 控制增強次數（以批次計算）
for i in range(125):  
    next(disgust_generator)  # 使用內建 next() 方法


Found 436 images belonging to 1 classes.


In [1]:
import os

train_path = 'fer2013/train'
class_counts = {cls: len(os.listdir(os.path.join(train_path, cls))) for cls in os.listdir(train_path)}

print("訓練集各類別樣本數量：", class_counts)


訓練集各類別樣本數量： {'angry': 3995, 'disgust': 4808, 'fear': 4097, 'happy': 7215, 'neutral': 4965, 'sad': 4830, 'surprise': 3171}


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

# 定義模型架構
model = Sequential([
    # 卷積層 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # 卷積層 2
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # 卷積層 3
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    
    # 全連接層
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 輸出 7 個情緒分類
])

# 編譯模型
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


c:\Users\hoisaline\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 訓練數據生成器（包括增強）
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,        # 像素值歸一化
    rotation_range=30,          # 隨機旋轉
    zoom_range=0.2,             # 隨機縮放
    shear_range=0.2,            # 剪切變換
    horizontal_flip=True,       # 水平翻轉
    fill_mode='nearest',        # 填充模式
    validation_split=0.1        # 劃分 10% 的數據作為驗證集
)

# 測試數據生成器（無增強）
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# 加載數據
train_generator = train_datagen.flow_from_directory(
    'fer2013/train',            # 訓練數據路徑
    target_size=(96, 96),
    color_mode='grayscale',     # 灰階圖片
    batch_size=64,
    class_mode='categorical',
    subset='training'           # 訓練集
)

val_generator = train_datagen.flow_from_directory(
    'fer2013/train',
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical',
    subset='validation'         # 驗證集
)

test_generator = test_datagen.flow_from_directory(
    'fer2013/test',             # 測試數據路徑
    target_size=(96, 96),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)


Found 29776 images belonging to 7 classes.
Found 3305 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# 早停
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

# 學習率調整
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)


In [7]:
# 訓練模型
history = model.fit(
    train_generator,
    epochs=50,                        # 訓練世代數
    validation_data=val_generator,    # 驗證數據
    callbacks=[early_stopping, reduce_lr]
)


c:\Users\hoisaline\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 0s 534ms/step - accuracy: 0.1967 - loss: 2.1592

c:\Users\hoisaline\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


466/466 ━━━━━━━━━━━━━━━━━━━━ 259s 553ms/step - accuracy: 0.1968 - loss: 2.1588 - val_accuracy: 0.2148 - val_loss: 3.5631 - learning_rate: 1.0000e-04
Epoch 2/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 223s 479ms/step - accuracy: 0.2345 - loss: 1.8950 - val_accuracy: 0.2475 - val_loss: 2.2514 - learning_rate: 1.0000e-04
Epoch 3/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 232s 498ms/step - accuracy: 0.2819 - loss: 1.8136 - val_accuracy: 0.2405 - val_loss: 3.0524 - learning_rate: 1.0000e-04
Epoch 4/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 205s 441ms/step - accuracy: 0.3464 - loss: 1.6188 - val_accuracy: 0.2590 - val_loss: 2.4189 - learning_rate: 1.0000e-04
Epoch 5/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 214s 459ms/step - accuracy: 0.3667 - loss: 1.5273 - val_accuracy: 0.2923 - val_loss: 2.1565 - learning_rate: 1.0000e-04
Epoch 6/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 206s 441ms/step - accuracy: 0.3820 - loss: 1.4907 - val_accuracy: 0.2905 - val_loss: 2.6180 - learning_rate: 1.0000e-04
Epoch 7/50
466/466 ━━━━━━━━━━━━━━━━━━━━ 220s 472ms/

In [ ]:
# 測試模型
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"測試集損失: {test_loss:.4f}")
print(f"測試集準確率: {test_accuracy:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# 可視化損失
plt.plot(history.history['loss'], label='訓練損失')
plt.plot(history.history['val_loss'], label='驗證損失')
plt.legend()
plt.title('訓練與驗證損失')
plt.show()

# 可視化準確率
plt.plot(history.history['accuracy'], label='訓練準確率')
plt.plot(history.history['val_accuracy'], label='驗證準確率')
plt.legend()
plt.title('訓練與驗證準確率')
plt.show()
